In [18]:
import cv2
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from matplotlib import pyplot as plt
model = load_model("final_model1.h5")

In [14]:
 # make a prediction for a new image.
# load and prepare the image
def load_image(filename):
    # load the image
    #img =cv2.cvtColor(filename, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(filename, (28, 28))
    # convert to array
    img = img_to_array(img)
    plt.imshow(img)
    plt.title('my picture')
    plt.show() 
    # reshape into a single sample with 1 channel
    img = img.reshape(1, 28, 28, 1)
    # prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img

In [3]:
# Read the image
img_for_box_extraction_path="/Users/ihjas/Desktop/fil/filledb.png"
img = cv2.imread(img_for_box_extraction_path, 0)
 
# Thresholding the image
(thresh, img_bin) = cv2.threshold(img, 128, 255,cv2.THRESH_BINARY|     cv2.THRESH_OTSU)# Invert the image
img_bin = 255-img_bin 
cv2.imwrite("Image_bin.jpg",img_bin)

True

In [4]:
# Defining a kernel length
kernel_length = np.array(img).shape[1]//200
 
# A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))# A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))# A kernel of (3 X 3) ones.
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

In [5]:
# Morphological operation to detect vertical lines from an image
img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
cv2.imwrite("verticle_lines.jpg",verticle_lines_img)# Morphological operation to detect horizontal lines from an image
img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)

True

In [6]:
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
cv2.imwrite("img_final_bin.jpg",img_final_bin)

True

In [16]:
import numpy as np
import argparse
import imutils
import cv2
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [17]:
# Find contours for image, which will detect all the boxes
im2, contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)# Sort all the contours by top to bottom.
(contours, boundingBoxes) = sort_contours(contours, method="top-to-bottom")

In [9]:
idx = 0
cropped_dir_path="/Users/ihjas/Desktop/2/"
for c in contours:
    # Returns the location and width,height for every contour
    x, y, w, h = cv2.boundingRect(c)
    if (w > 10 and h > 10 and w <25 and h < 25) :
        idx += 1
        new_img = img[y+2:y+h-2, x+2:x+w-2]
        rec=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        #cv2.imwrite(cropped_dir_path+str(idx) + '.png', new_img)# If the box height is greater then 20, widht is >80, then only save it as a box in "cropped/" folder.
    if (w > 10 and h > 10 and w <25 and h < 25) :
        idx += 1
        new_img = img[y+2:y+h-2, x+2:x+w-2]
        rec=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        
        config = ('-l eng --oem 1 --psm 3')
        new_img=load_image(new_img)
        text = model.predict_classes(new_img)
        digi=str(text[0])
        cv2.putText(img,digi, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .2, (36,255,12), 1)
        cv2.imwrite(cropped_dir_path+str(idx) + '.png', new_img)

In [10]:
from matplotlib import pyplot as plt
plt.figure(figsize=(50,50))
plt.imshow(img)
plt.title('my picture')
plt.show()

True